# Import Section

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install Biopython

     |████████████████████████████████| 2.3MB 31.3MB/s 


In [2]:
import json,os,csv,networkx as nx,numpy as np,math

In [3]:
path='/content/drive/MyDrive/NPPI/'

In [4]:

with open(path+'Gold-DIP.json','r') as f1:
  POSL=json.load(f1)
with open(path+'GoldNeg.json','r') as f1:
  NEGL=json.load(f1)
print(len(POSL[:2000]),len(NEGL[2000]))

FileNotFoundError: ignored

In [5]:
DKL=set()
for k in POSL:
  DKL.add(k[0]);DKL.add(k[1])
for k in NEGL:
  DKL.add(k[0]);DKL.add(k[1])
print(len(DKL))
f=open(path+"UniqueProtPPI.txt",'w')
f.write("\n".join(list(DKL)));f.close()

NameError: ignored

In [8]:
import json,csv
from Bio import SeqIO
import numpy as np
import networkx as nx
from itertools import permutations
import json,csv,random
import numpy as np, math


In [32]:
from sklearn.svm import SVR
from sklearn import svm
from scipy import interp
import matplotlib.pyplot as plt_s
import matplotlib.pyplot as plt_r
from itertools import cycle
from sklearn.metrics import roc_curve, auc,accuracy_score, matthews_corrcoef
from collections import defaultdict
from sklearn.metrics import precision_recall_curve,f1_score
from sklearn.ensemble import RandomForestClassifier
#from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
import time,datetime
import math
################
from sklearn.model_selection import StratifiedKFold as SFK
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


# Parsing Fasta 

In [10]:
ProtSeq={};s='';PID=[]
fasta_sequences = SeqIO.parse(open(path+"FastaSeq/Protein.fasta"),'fasta')
for fasta in fasta_sequences:
    sequence = str(fasta.seq)#fasta.seq.tostring()
    name=(fasta.id).split("\n")[0].split("|")[1]
    #if name in ExList:
    ProtSeq[name]=sequence
    PID.append(name)
#with open('./Prediction/FuzzyPPI_Data.json','w') as fp:
    #json.dump(ProtSeq,fp)
print("----------------------------------")
print("Length of existing Protein with Seq={}".format(len(ProtSeq)))

----------------------------------
Length of existing Protein with Seq=2883


# Sequence Processing

In [ ]:
global TrProtSeq
def SeqTransform(ProtSeq):
    """Multivariate PAper"""
    FasD={"A":"#","G":"#","V":"#","C":"$","D":"&","E":"&","F":"@","I":"@","L":"@","P":"@",\
          "H":"*","N":"*","Q":"*","W":"*","K":"+","R":"+","M":"?","S":"?","T":"?","Y":"?"}
    ''' Weighted Sparse Representation'''
   # FasD={"A":"#","L":"#","V":"#","I":"#","M":"#","C":"#",\
   #       "F":"@","W":"@","Y":"@",\
   #       "S":"?","T":"?","N":"?","Q":"?","H":"?",\
   #       "K":"+","R":"+",\
   #       "D":"&","E":"&",\
   #       "G":"*","P":"*"}
    TrProtSeq={}
    for pid in ProtSeq:
        seq=ProtSeq[pid].replace("U","")
        EncSeq1=[str(FasD[x]) for x in seq]
        TrProtSeq[pid]="".join(EncSeq1)
    print("SEQ Processed")
    return TrProtSeq


TrProtSeq= SeqTransform(ProtSeq)
with open(path+'PPI_Encoded-SEQ.json','w') as fp:
    json.dump(TrProtSeq,fp)

print("Length of existing Protein with Seq={}".format(len(TrProtSeq)))


SEQ Processed
Length of existing Protein with Seq=2883


# Bigram and Trigram

In [ ]:
def genRedSeqFeatT():    
    global Feature
    #A=["#","$","&","@","*","+","?"]#A=["A","B","C","D","E","F","G"]
    TiAA=[];
    A=["#","$","@","&","*","+","?"]
    for i,a in enumerate(A):
        for j,b in enumerate(A):
            for k,c in enumerate(A):
                if i<=j and j<=k:
                    TiAA.append([a,b,c])
    print(len(TiAA))
    #TiAA[0]
    return TiAA
def genRedSeqFeatB():   
    global Feature
    A=["#","$","@","&","*","+","?"]
    BiAAq=[]
    for i,a in enumerate(A):
        for j,b in enumerate(A):
            if i<=j:
                BiAAq.append(a+b)
    print(len(BiAAq))
    return BiAAq  
TiAA= genRedSeqFeatT()
BiAAq= genRedSeqFeatB()

84
28


# Unigram Fetaure

In [ ]:
def Unigram_Freq(ln,sq):  
    Dl={}
    A=["#","$","@","&","*","+","?"]
    for a in A:
        Dl[a]=(sq.count(a)+1)/(ln+1)
        
    #print(d)   
    #with open('./Prediction/Result/Unigram_frequency.json','a') as fp:
            #json.dump(d,fp)
    return Dl 
#dic={}
#f=open('./Prediction/Uniprot_freq.json','w')
with open(path+'PPI_Encoded-SEQ.json','r') as fp:
    seq=json.load(fp)
UFQ={}#Unigram Frequency
for d in seq:
    sq=seq[d]
    UFQ[d]=Unigram_Freq(len(sq),sq)
print(len(UFQ))

2883


# Bigram and Trigram frequency

In [ ]:
def Bigram_Freq(ln,sq):  
    Db={}
    global BiAAq
    for bi in BiAAq:
        ib=bi[::-1]
        if not(bi in Db or ib in Db):
            if bi==ib:
                Db[bi]=(sq.count(bi)+1)/(ln+1)
            else:
                 Db[bi]=(sq.count(bi)+1)/(ln+1)+(sq.count(ib)+1)/(ln+1)
    return Db
def Trigram_Freq(ln,sq):  
    Dt={};D={}
    global TiAA
    for i,a in enumerate(TiAA):
        D[''.join(a)]=list(set([''.join(x) for x in list(permutations(a))]))
    for d in D:
        c=0
        for ls in D[d]:
            c+=(sq.count(ls)+1)/(ln+1)
        Dt[d]=c
    return Dt
TFQ={};BFQ={}
with open(path+'PPI_Encoded-SEQ.json','r') as fp:
    seq=json.load(fp)
for d in seq:
    sq=seq[d]
    ln=len(seq[d])
    BFQ[d]=Bigram_Freq(len(sq),sq)
    TFQ[d]=Trigram_Freq(len(sq),sq)
print(len(TFQ))
print(len(BFQ))
D={};
for i,a in enumerate(TiAA):
    D[''.join(a)]=list(set([''.join(x) for x in list(permutations(a))]))
print(len(D))

2883
2883
84


In [ ]:
def getFet(R):
  a,b=k
  d1=[TFQ[a][x] for x in TFQ[a]]+[TFQ[b][x] for x in TFQ[b]]
  


In [ ]:
d1=[TFQ[a][x] for x in TFQ[a]]+[TFQ[b][x] for x in TFQ[b]]
print(d1)

[0.0059136605558840925, 0.006505026611472502, 0.020697811945594325, 0.014192785334121822, 0.010053222945002958, 0.01596688350088705, 0.0254287403903016, 0.001774098166765228, 0.010644589000591367, 0.00827912477823773, 0.00768775872264932, 0.007096392667060911, 0.009461856889414548, 0.02069781194559432, 0.039030159668835, 0.027202838557066823, 0.03015966883500887, 0.04790065050266115, 0.013010053222945003, 0.01655824955647546, 0.029568302779420463, 0.027202838557066823, 0.010053222945002957, 0.019515079834417505, 0.02661147250147842, 0.012418687167356595, 0.029568302779420463, 0.02069781194559432, 0.0005913660555884093, 0.0029568302779420462, 0.001774098166765228, 0.001774098166765228, 0.001774098166765228, 0.002365464222353637, 0.009461856889414548, 0.009461856889414548, 0.008870490833826138, 0.010644589000591367, 0.012418687167356595, 0.004139562389118865, 0.004730928444707274, 0.005322294500295683, 0.004730928444707274, 0.004139562389118865, 0.006505026611472502, 0.005913660555884092

# Feature Bigram

In [ ]:
D1=[];L1=[]
for k in POSL[:2000]:
  a,b=k
  dx=[TFQ[a][x] for x in TFQ[a]]+[BFQ[a][x] for x in BFQ[a]]+[TFQ[b][x] for x in TFQ[b]]+[BFQ[b][x] for x in BFQ[b]]
  D1.append(dx);L1.append(1)
for k in NEGL[:2000]:
  a,b=k
  dx=[TFQ[a][x] for x in TFQ[a]]+[BFQ[a][x] for x in BFQ[a]]+[TFQ[b][x] for x in TFQ[b]]+[BFQ[b][x] for x in BFQ[b]]
  D1.append(dx);L1.append(1)
D=np.array(D1);L=np.array(L1)
print(D.shape,L.shape)

(4000, 224) (4000,)


In [ ]:
with open(path+'FetData-2.json','w') as fp:
  json.dump(D1,fp)
with open(path+'FetLabel-2.json','w') as fp:
  json.dump(L1,fp)

# MMI Feature

In [ ]:
def IBi(fa,fb,fab):  # I(a,b)=f(a,b) ln (f(a,b)/(f(a)*f(b)))
    return(fab*(math.log(fab)/(fa*fb)))
    

def CMI(fc,fac,fbc,fabc):# I(a,b|c)=H(a|c)-H(a|b,c)    
    Ha_c=-(fac/fc)*math.log(fac/fc) # H(a|c)
    Ha_bc=-(fabc/fbc)*math.log(fabc/fbc) # H(a|b,c)
    Ia_bc=Ha_c-Ha_bc
    return Ia_bc
def getFet(p):
    global UFQ,BFQ,TFQ
    A=["#","$","@","&","*","+","?"]
    FrqU={};I3={};UF=UFQ[p];BF=BFQ[p];TF=TFQ[p]
    for fet in A:
        I3[fet]=UF[fet]
    for  key in BiAAq:
        a=key[0];b=key[1]
        fa=UF[a];fb=UF[b]
        fab=0;fac=0;fbc=0;
        ab="{}{}".format(a,b);ba="{}{}".format(b,a)
        if ab in BiAAq:fab=BF[ab]
        else:fab=BF[ba]
        Iab=IBi(fa,fb,fab)  #IBi(key[:2],FrqB,FrqU,U) 
        I3[key]=Iab
    for  key in D:     
        a=key[0];b=key[1];c=key[2]
        fa=UF[a];fb=UF[b];fc=UF[c]#k
        fab=0;fac=0;fbc=0;
        ab="{}{}".format(a,b);ba="{}{}".format(b,a)
        if ab in BiAAq:fab=BF[ab]
        else:fab=BF[ba]
            
        ac="{}{}".format(a,c);ba="{}{}".format(c,a)
        if ac in BiAAq:fac=BF[ac]
        else:fac=BF[ca]
            
        bc="{}{}".format(b,c);ba="{}{}".format(c,b)
        if bc in BiAAq:fbc=BF[bc]
        else:fbc=BF[cb]
        fabc=TF[key]

        Iab=IBi(fa,fb,fab)  #IBi(key[:2],FrqB,FrqU,U) 
        Ia_bc=CMI(fc,fac,fbc,fabc)
        Iabc=Iab-Ia_bc
        I3[key]=Iabc
    return I3

In [ ]:
PI={}
for p in seq:
    PI[p]=getFet(p)
print(len(PI))  
with open(path+'MMI_Fet.json','w') as fp:
    json.dump(PI,fp)

2883


In [ ]:
for p in PI:
  print(p,PI[p])
  print(len([PI[p][x] for x in PI[p]]))
  break

O00327 {'#': 0.17862838915470494, '$': 0.023923444976076555, '@': 0.24082934609250398, '&': 0.12599681020733652, '*': 0.10047846889952153, '+': 0.11802232854864433, '?': 0.22169059011164274, '##': -3.565108112484802, '#$': -9.015940207978154, '#@': -5.118814532587849, '#&': -6.605590937692619, '#*': -6.755658726552308, '#+': -6.7432662731279365, '#?': -4.7080299038055, '$$': -17.94877102656374, '$@': -7.721961043101592, '$&': -12.78209244675384, '$*': -23.15059161349066, '$+': -11.42078168127809, '$?': -14.275896469408721, '@@': -2.5633204077323057, '@&': -5.503716501023536, '@*': -6.901485771124752, '@+': -5.667125346769668, '@?': -4.584624684112274, '&&': -3.8370971060540433, '&*': -8.985476424152974, '&+': -7.649797496238342, '&?': -5.548072754569968, '**': -3.8162709875039815, '*+': -6.776759553845979, '*?': -5.925182224799781, '++': -5.092206828753633, '+?': -5.558786863721573, '??': -2.959354157664722, '###': -3.600991529628731, '##$': -3.7527973676561373, '##@': -3.5648015296057

In [ ]:
def DataProc(DT):
  D2=[];L2=[]
  for k in DT:
    a,b,c=k
    dx=[PI[a][x] for x in PI[a]]+[PI[b][x] for x in PI[b]]
    if int(c)==1:
      L2.append(1)
    else:
      L2.append(0)
    D2.append(dx);

  
  #D=np.array(D2);L=np.array(L2)
  #print(D.shape,L.shape)
  return D2,L2

In [23]:
def LoadIntr(path,DSet,fname):
  pathx=path+'DS/'+DSet
  with open(pathx+'/{}'.format(fname),'r') as fp:
    Data=json.load(fp)
  return Data

In [ ]:
def writedata(path,DSet,fname,D2,L2):
  pathx=path+'DS/'+DSet
  with open(pathx+'/RD-{}'.format(fname),'w') as fp:
    json.dump(D2,fp)
  with open(pathx+'/RL-{}'.format(fname),'w') as fp:
    json.dump(L2,fp)

In [ ]:
DSetL=["DS20","DS100"]
for DSet in DSetL:
  for i in range(1,4):
    fnameL=['A','B']
    for fni in fnameL:
      fname='DS{}{}.json'.format(i,fni)
      
      DT=LoadIntr(path,DSet,fname)
      D,L=DataProc(DT)
      writedata(path,DSet,fname,D,L)

In [25]:
DT=LoadIntr(path,"DS20","DS1A.json")

In [27]:
L=[]
for d in DT:
  #print(d)
  L.append(d[2])
print(len(L),sum(L))

6710 3355


In [28]:
3355*2

6710

In [11]:
def LoadData(path,DSet,fname):
  pathx=path+'DS/'+DSet
  with open(pathx+'/RD-{}'.format(fname),'r') as fp:
    Data=json.load(fp)
  with open(pathx+'/RL-{}'.format(fname),'r') as fp:
    Label=json.load(fp)
  
  D=np.array(Data)
  L=np.array(Label)
  print("Data Dimension {}".format(D.shape))
  print("Label Dimension {}".format(D.shape))
  return D,L

In [43]:
classifiersX = {
  'SVM' : svm.SVC(C=2.0, kernel='rbf',gamma=0.0005, probability=True),#svm.SVC(kernel='linear', C=0.01),
  'RF' : RandomForestClassifier(n_estimators=192,min_samples_leaf = 75),  # random_forest
  #'decision_tree' : DecisionTreeClassifier(random_state=0),
  'KNN' : KNeighborsClassifier(n_neighbors=7),  # kneighbors
}

In [59]:
def Classify(X,y,DSet,fname):
  #cross validation and classification
  kf = SFK(n_splits=5)
  i=0
  S=""
  for train_index, test_index in kf.split(X,y):

     #print("TRAIN:", train_index, "TEST:", test_index)
     Xtrd, Xtst = X[train_index], X[test_index]
     Ltrd, Ltst = y[train_index], y[test_index]
     print(i,Xtrd.shape, Xtst.shape)
     print(Ltrd.shape, Ltst.shape)
     print(sum(Ltrd),sum(Ltst))
     i+=1
     clf=classifiersX['RF']
     clf.fit(Xtrd, Ltrd)
     prediction = clf.predict(Xtst)
     probas_RF = clf.predict_proba(Xtst)
     AUC=roc_auc_score(Ltst,probas_RF[:, 1])
     ACCU=accuracy_score(Ltst,prediction)
     preRF, recRF, thrRF = precision_recall_curve(Ltst, probas_RF[:,1])
     AUPRC = auc(recRF, preRF)
     f1_s=f1_score(Ltst, prediction)
     mcc_s=matthews_corrcoef(Ltst, prediction)
     #DScore={"AUC":AUC, "ACCU": ACCU, "AUPRC": AUPRC, "F1": f1_s, "MCC": mcc_s }

     S+="{},{},{},{},{},{},{},{}\n".format(DSet,fname, i, AUC, ACCU,AUPRC,f1_s,mcc_s)

    
     f=open(path+'Result.csv',"a")
     f.write(S);f.close()

     #AUC,AUPRC, precision recall, accuracy, f1,mcc
     #DSet number reqd, fname, fold number(i)

In [61]:
S="DSet,Fname, fold, AUC, ACCU, AUPRC, f1_s, mcc_s \n"
f=open(path+'Result.csv',"w")
f.write(S);f.close()
DSetL=["DS20","DS100"]
for DSet in DSetL:
  for i in range(1,4):
    fnameL=['A','B']
    for fni in fnameL:
      fname='DS{}{}.json'.format(i,fni)
      DX,LX=LoadData(path,DSet,fname)
      Classify(DX,LX,DSet,'DS{}{}'.format(i,fni))
      break
    break
  break

Data Dimension (6710, 238)
Label Dimension (6710, 238)
0 (5368, 238) (1342, 238)
(5368,) (1342,)
2684 671
1 (5368, 238) (1342, 238)
(5368,) (1342,)
2684 671
2 (5368, 238) (1342, 238)
(5368,) (1342,)
2684 671
3 (5368, 238) (1342, 238)
(5368,) (1342,)
2684 671
4 (5368, 238) (1342, 238)
(5368,) (1342,)
2684 671


In [22]:
Data Dimension (6710, 238)
Label Dimension (6710, 238)
0 (5368, 238) (1342, 238)
(5368,) (1342,)
2013 1342
1 (5368, 238) (1342, 238)
(5368,) (1342,)
2013 1342
2 (5368, 238) (1342, 238)
(5368,) (1342,)
2684 671
3 (5368, 238) (1342, 238)
(5368,) (1342,)
3355 0
4 (5368, 238) (1342, 238)
(5368,) (1342,)
3355 0

6710

In [ ]:
X,Y=LoadData(path,1)  # path -path to the json file, dset is dataset number, here it is 1(group 1) and 2(group 2)

Data Dimension (4000, 238)
Label Dimension (4000, 238)


In [ ]:
X,Y=LoadData(path,2)  # path -path to the json file, dset is dataset number, here it is 1(group 1) and 2(group 2)

Data Dimension (4000, 224)
Label Dimension (4000, 224)


# ---------------------------------------------------------------------------------------------------------------

# NMBAC Feature

# Unigram of Sequence

In [ ]:
def UniGram(seq):
    global Let
    Un=[]
    for a in Let:
        Un.append(seq.count(a))
    return Un

# NMBAC Computation

In [ ]:
def Comp_nmbac(seq,lag):
    global D,Let,Prop
    Let=['A', 'C' ,'D' ,'E', 'F', 'G', 'H', 'I' ,'K', 'L' ,'M' ,'N', 'P' ,'Q','R','S','T','V', 'W' ,'Y']
    s1='';nt=1/(len(seq)-lag)
    H=0;VSC=0;P1=0;P2=0;SASA=0;NCISC=0
    for i in range(int(len(seq)/lag)):
        sub=seq[i*lag:(i+1)*lag]
        for ax in Let:
            H+=sub.count(ax)*D[ax]['H'];VSC+=sub.count(ax)*D[ax]['VSC'];P1+=sub.count(ax)*D[ax]['P1']
            P2+=sub.count(ax)*D[ax]['P2'];SASA+=sub.count(ax)*D[ax]['SASA'];NCISC+=sub.count(ax)*D[ax]['NCISC']
    return{'H': H*nt, 'VSC': VSC*nt, 'P1': P1*nt, 'P2': P2*nt, 'SASA': SASA*nt, 'NCISC': NCISC*nt}

In [ ]:
with open('./prediction/phyProp_6.json', 'r') as f:
    D=json.load(f) 
Let=['A', 'C' ,'D' ,'E', 'F', 'G', 'H', 'I' ,'K', 'L' ,'M' ,'N', 'P' ,'Q','R','S','T','V', 'W' ,'Y']
Prop=["H","VSC","P1","P2","SASA","NCISC"]
with open('./Prediction/FuzzyPPI_Data.json','r') as fp:
    Plist=json.load(fp)
PMD={}
for protein in Plist:#Plist is a dic. of protein seq. with pid
    seq=Plist[protein]
    PMD[protein]={}
    PMD[protein]={"Phy":{},"UniG":[]}
    for i in range(1,31):
        PMD[protein]['Phy'][i]=Comp_nmbac(seq,i)
    PMD[protein]["UniG"]=UniGram(seq)

#with open('./prediction/Result/NMBC_Fet.json', 'w') as f:
    #json.dump(PMD,f) 

# Merging two features

# Loading MMI feature

In [ ]:
FET={}
with open('./Prediction/Result/MMI_Fet.json','r') as fp:
    MMI=json.load(fp)

# Feature Merging

In [ ]:
def fet(pid):
    global MMI
    global PMD
    fetl=[]
    MM=MMI[pid];UF=PMD[pid]['UniG'];PHY=PMD[pid]['Phy']
    for m in MM:
        fetl.append(MM[m])
    #/for p in PHY:
    for i in range(1,31):
        for j in PHY[i]:
            fetl.append(PHY[i][j])
            
    for u in UF:
        fetl.append(u)
    #FET[pid]=fetl
    #print(len(fetl))
    return fetl
#D={}
#D['O43930']=fet('O43930')  
#print(len(D))

In [ ]:
D={}
with open('./prediction/FuzzyPPI_Data.json', 'r') as f:
    dt=json.load(f) 
for d in dt:
    D[d]=fet(d)
with open("./prediction/Result/Merged_feature.json","w") as fd:
    json.dump(D,fd)